# Customer Churn Data Set Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [2]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML201EN-SkillsNetwork/labs/module_4/datasets/im_churn.csv', index_col=False)

In [3]:
df.head()

,tenure,MonthlyCharges,TotalCharges,Partner_0,Partner_1,Dependents_0,Dependents_1,PhoneService_0,PhoneService_1,MultipleLines_0,...,Contract_0,Contract_1,Contract_2,PaperlessBilling_0,PaperlessBilling_1,PaymentMethod_0,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3,Class
0,27,70.55,1943.90,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
1,69,93.30,6398.05,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0
2,55,59.20,3175.85,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0
3,49,59.60,2970.30,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
4,72,109.55,7887.25,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6174 entries, 0 to 6173
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tenure              6174 non-null   int64  
 1   MonthlyCharges      6174 non-null   float64
 2   TotalCharges        6174 non-null   float64
 3   Partner_0           6174 non-null   float64
 4   Partner_1           6174 non-null   float64
 5   Dependents_0        6174 non-null   float64
 6   Dependents_1        6174 non-null   float64
 7   PhoneService_0      6174 non-null   float64
 8   PhoneService_1      6174 non-null   float64
 9   MultipleLines_0     6174 non-null   float64
 10  MultipleLines_1     6174 non-null   float64
 11  MultipleLines_2     6174 non-null   float64
 12  InternetService_0   6174 non-null   float64
 13  InternetService_1   6174 non-null   float64
 14  InternetService_2   6174 non-null   float64
 15  OnlineSecurity_0    6174 non-null   float64
 16  Online

In [6]:
df.shape

(6174, 43)

In [7]:
df.shape[0] * df.shape[1]

265482